# Обработка выбросов разными способами 

In [1]:
from python_files.data import data
from python_files.strength.str_col_list import str_col_list_norm

In [2]:
import numpy as np
import pandas as pd

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
data_ = data.copy()

##### Признак 'Угол нашивки, град', имеющий всего два уникальных значения, обрабатывать на выбросы не имеет смысла. Остальные признаки, за исключением целевого, будем обрабатывать исходя из идеи сохранения их исходного распределения.

Объявим необходимые пользовательские функции.

In [5]:
# Тип вычисления верхней и нижней границы выбросов
from enum import Enum
class OutlierBoundaryType(Enum):
    SIGMA = 1
    QUANTILE = 2
    IQR = 3

In [6]:
# Функция вычисления верхней и нижней границы выбросов
def get_outlier_boundaries(df, col, outlier_boundary_type: OutlierBoundaryType):
    if outlier_boundary_type == OutlierBoundaryType.SIGMA:
        K1 = 3
        lower_boundary = df[col].mean() - (K1 * df[col].std())
        upper_boundary = df[col].mean() + (K1 * df[col].std())

    elif outlier_boundary_type == OutlierBoundaryType.QUANTILE:
        lower_boundary = df[col].quantile(0.025)
        upper_boundary = df[col].quantile(0.975)

    elif outlier_boundary_type == OutlierBoundaryType.IQR:
        K2 = 1.5
        IQR = df[col].quantile(0.75) - df[col].quantile(0.25)
        lower_boundary = df[col].quantile(0.25) - (K2 * IQR)
        upper_boundary = df[col].quantile(0.75) + (K2 * IQR)

    else:
        raise NameError('Unknown Outlier Boundary Type')
        
    return lower_boundary, upper_boundary    

#### 1. Удаление выбросов 

In [7]:
# Разделим исходный датасет на обучающую и тестовую выборки и будем работать с выбросами только на обучающей выборке
y_str = data_.pop('Прочность при растяжении, МПа')
X_str = data_

In [10]:
X_train_str, X_test_str, y_train_str, y_test_str = train_test_split(X_str, y_str,
                                                    test_size = 0.3,
                                                    random_state = 7)

In [12]:
# соберем всю обучающую выборку в новую переменную df_train 
df_train_str = X_train_str
df_train_str['Прочность при растяжении, МПа'] = y_train_str

##### 1.1 Удаление выбросов определенных на основе межквартильного интервала в признаке со скошенным исходным распределением

In [49]:
# сначала обработаем выбросы в признаке со скошенным исходным распределением ('Поверхностная плотность, г/м2')
# Вычисление по межквартильному интервалу верхней и нижней границы в признаке со скошенным исходным распределением  
lower_boundary_, upper_boundary_ = get_outlier_boundaries(df_train_str, 'Поверхностная плотность, г/м2', OutlierBoundaryType.IQR)

# Флаги для удаления выбросов в признаке со скошенным исходным распределением 
outliers_ = np.where(df_train_str['Поверхностная плотность, г/м2'] > upper_boundary_, True, 
                     np.where(df_train_str['Поверхностная плотность, г/м2'] < lower_boundary_, True, False))

# Удаление выбросов на основе флага в признаке со скошенным исходным распределением
df_train_str_trimmed = df_train_str.loc[~(outliers_), ]

##### 1.2 Удаление выбросов определенных по правилу "3 сигм" в признаках с исходным распределением близким к нормальному 

In [50]:
# удалим выбросы в признаках с исходным распределением близким к нормальному  
for col in str_col_list_norm:

    # Вычисление по правилу "3 сигм" верхней и нижней границы 
    lower_boundary, upper_boundary = get_outlier_boundaries(df_train_str_trimmed, col, OutlierBoundaryType.SIGMA)
    
    # Флаги для удаления выбросов 
    outliers = np.where(df_train_str_trimmed[col] > upper_boundary, True, 
                        np.where(df_train_str_trimmed[col] < lower_boundary, True, False))
    
    # Удаление выбросов на основе флага
    df_train_str_trimmed_3S = df_train_str_trimmed.loc[~(outliers), ]

In [51]:
# выделим целевой признак в отдельную переменную после обработки выбросов
y_train_str_trimmed_3S = df_train_str_trimmed_3S.pop('Прочность при растяжении, МПа')
X_train_str_trimmed_3S = df_train_str_trimmed_3S

# формируем DataFrame на основе массива
y_train_str_trimmed_3S = pd.DataFrame(y_train_str_trimmed_3S, columns=['Прочность при растяжении, МПа'])

##### 1.3 Удаление выбросов определенных по квантилям (2.5% и 97.5%) в признаках с исходным распределением близким к нормальному 

In [52]:
# удалим выбросы в признаках с исходным распределением близким к нормальному  
for col in str_col_list_norm:

    # Вычисление по квантилям (2.5% и 97.5%) верхней и нижней границы 
    lower_boundary, upper_boundary = get_outlier_boundaries(df_train_str_trimmed, col, OutlierBoundaryType.QUANTILE)
    
    # Флаги для удаления выбросов 
    outliers = np.where(df_train_str_trimmed[col] > upper_boundary, True, 
                        np.where(df_train_str_trimmed[col] < lower_boundary, True, False))
    
    # Удаление выбросов на основе флага
    df_train_str_trimmed_QT = df_train_str_trimmed.loc[~(outliers), ]

In [53]:
# выделим целевой признак в отдельную переменную после обработки выбросов
y_train_str_trimmed_QT = df_train_str_trimmed_QT.pop('Прочность при растяжении, МПа')
X_train_str_trimmed_QT = df_train_str_trimmed_QT

# формируем DataFrame на основе массива
y_train_str_trimmed_QT = pd.DataFrame(y_train_str_trimmed_QT, columns=['Прочность при растяжении, МПа'])

#### 2. Замена выбросов граничными значениями

##### 2.1 Замена выбросов определенных на основе межквартильного интервала в признаке со скошенным исходным распределением

In [54]:
df_train_str_sub = df_train_str.copy()     
# Вычисление по межквартильному интервалу верхней и нижней границы в признаке со скошенным исходным распределением
lower_boundary, upper_boundary = get_outlier_boundaries(df_train_str_sub, 'Поверхностная плотность, г/м2', OutlierBoundaryType.IQR)
# Изменение данных
df_train_str_sub['Поверхностная плотность, г/м2'] = np.where(df_train_str_sub['Поверхностная плотность, г/м2'] > upper_boundary, upper_boundary, 
       np.where(df_train_str_sub['Поверхностная плотность, г/м2'] < lower_boundary, lower_boundary, df_train_str_sub['Поверхностная плотность, г/м2']))

##### 2.2 Замена выбросов определенных по правилу "3 сигм" в признаках с исходным распределением близким к нормальному 

In [55]:
# Замена выбросов граничными значениями в признаках с исходным распределением близким к нормальному  
for col in str_col_list_norm:    
    # Вычисление верхней и нижней границы
    lower_boundary, upper_boundary = get_outlier_boundaries(df_train_str_sub, col, OutlierBoundaryType.SIGMA)
    # Изменение данных
    df_train_str_sub[col] = np.where(df_train_str_sub[col] > upper_boundary, upper_boundary, 
                        np.where(df_train_str_sub[col] < lower_boundary, lower_boundary, df_train_str_sub[col]))
# cохраним результат в новую переменную
df_train_str_sub_bound_3S = df_train_str_sub

In [56]:
# выделим целевой признак в отдельную переменную после обработки выбросов
y_train_str_sub_bound_3S = df_train_str_sub_bound_3S.pop('Прочность при растяжении, МПа')
X_train_str_sub_bound_3S = df_train_str_sub_bound_3S

# формируем DataFrame на основе массива
y_train_str_sub_bound_3S = pd.DataFrame(y_train_str_sub_bound_3S, columns=['Прочность при растяжении, МПа'])

##### 2.3 Замена выбросов определенных по квантилям (2.5% и 97.5%) в признаках с исходным распределением близким к нормальному 

In [57]:
# повторяем обработку выбросов для признака 'Поверхностная плотность, г/м2' сохранив результат в нлвую переменную 'df_train_sub_'
df_train_str_sub_ = df_train_str.copy()     
# Вычисление по межквартильному интервалу верхней и нижней границы в признаке со скошенным исходным распределением
lower_boundary, upper_boundary = get_outlier_boundaries(df_train_str_sub_, 'Поверхностная плотность, г/м2', OutlierBoundaryType.IQR)
# Изменение данных
df_train_str_sub_['Поверхностная плотность, г/м2'] = np.where(df_train_str_sub_['Поверхностная плотность, г/м2'] > upper_boundary, upper_boundary, 
    np.where(df_train_str_sub_['Поверхностная плотность, г/м2'] < lower_boundary, lower_boundary, df_train_str_sub_['Поверхностная плотность, г/м2']))

In [58]:
# Замена выбросов граничными значениями в признаках с исходным распределением близким к нормальному  
for col in str_col_list_norm:    
    # Вычисление верхней и нижней границы
    lower_boundary, upper_boundary = get_outlier_boundaries(df_train_str_sub_, col, OutlierBoundaryType.QUANTILE)
    # Изменение данных
    df_train_str_sub_[col] = np.where(df_train_str_sub_[col] > upper_boundary, upper_boundary, 
                        np.where(df_train_str_sub_[col] < lower_boundary, lower_boundary, df_train_str_sub_[col]))
# cохраним результат в новую переменную
df_train_str_sub_bound_QT = df_train_str_sub_

In [59]:
# выделим целевой признак в отдельную переменную после обработки выбросов
y_train_str_sub_bound_QT = df_train_str_sub_bound_QT.pop('Прочность при растяжении, МПа')
X_train_str_sub_bound_QT = df_train_str_sub_bound_QT

# формируем DataFrame на основе массива
y_train_str_sub_bound_QT = pd.DataFrame(y_train_str_sub_bound_QT, columns=['Прочность при растяжении, МПа'])

#### 3. Замена выбросов по медиане

##### 3.1 Замена выбросов определенных на основе межквартильного интервала в признаке со скошенным исходным распределением

In [60]:
# В признаке со скошенным исходным распределением заменим выбросы по моде
df_train_str_sub_med = df_train_str.copy()     
# Вычисление по межквартильному интервалу верхней и нижней границы в признаке со скошенным исходным распределением
lower_boundary, upper_boundary = get_outlier_boundaries(df_train_str_sub_med, 'Поверхностная плотность, г/м2', OutlierBoundaryType.IQR)
# Изменение данных
df_train_str_sub_med['Поверхностная плотность, г/м2'] = np.where(df_train_str_sub_med['Поверхностная плотность, г/м2'] > upper_boundary,
                                                             df_train_str_sub_med['Поверхностная плотность, г/м2'].mode(), 
        np.where(df_train_str_sub_med['Поверхностная плотность, г/м2'] < lower_boundary, df_train_str_sub_med['Поверхностная плотность, г/м2'].mode(),
                    df_train_str_sub_med['Поверхностная плотность, г/м2']))

##### 3.2 Замена выбросов определенных по правилу "3 сигм" в признаках с исходным распределением близким к нормальному 

In [61]:
# Замена выбросов граничными значениями в признаках с исходным распределением близким к нормальному  
for col in str_col_list_norm:    
    # Вычисление верхней и нижней границы
    lower_boundary, upper_boundary = get_outlier_boundaries(df_train_str_sub_med, col, OutlierBoundaryType.SIGMA)
    # Изменение данных
    df_train_str_sub_med[col] = np.where(df_train_str_sub_med[col] > upper_boundary, df_train_str_sub_med[col].median(), 
                        np.where(df_train_str_sub_med[col] < lower_boundary, df_train_str_sub_med[col].median(), df_train_str_sub_med[col]))
# cохраним результат в новую переменную
df_train_str_sub_med_3S = df_train_str_sub_med

In [62]:
# выделим целевой признак в отдельную переменную после обработки выбросов
y_train_str_sub_med_3S = df_train_str_sub_med_3S.pop('Прочность при растяжении, МПа')
X_train_str_sub_med_3S = df_train_str_sub_med_3S

# формируем DataFrame на основе массива
y_train_str_sub_med_3S = pd.DataFrame(y_train_str_sub_med_3S, columns=['Прочность при растяжении, МПа'])

##### 3.3 Замена выбросов определенных по квантилям (2.5% и 97.5%) в признаках с исходным распределением близким к нормальному 

In [63]:
# В признаке со скошенным исходным распределением заменим выбросы по моде
df_train_str_sub_med_ = df_train_str.copy()     
# Вычисление по межквартильному интервалу верхней и нижней границы в признаке со скошенным исходным распределением
lower_boundary, upper_boundary = get_outlier_boundaries(df_train_str_sub_med_, 'Поверхностная плотность, г/м2', OutlierBoundaryType.IQR)
# Изменение данных
df_train_str_sub_med_['Поверхностная плотность, г/м2'] = np.where(df_train_str_sub_med_['Поверхностная плотность, г/м2'] > upper_boundary,
                                                             df_train_str_sub_med_['Поверхностная плотность, г/м2'].mode(), 
        np.where(df_train_str_sub_med_['Поверхностная плотность, г/м2'] < lower_boundary, df_train_str_sub_med_['Поверхностная плотность, г/м2'].mode(),
                    df_train_str_sub_med_['Поверхностная плотность, г/м2']))

In [64]:
# Замена выбросов граничными значениями в признаках с исходным распределением близким к нормальному  
for col in str_col_list_norm:    
    # Вычисление верхней и нижней границы
    lower_boundary, upper_boundary = get_outlier_boundaries(df_train_str_sub_med_, col, OutlierBoundaryType.QUANTILE)
    # Изменение данных
    df_train_str_sub_med_[col] = np.where(df_train_str_sub_med_[col] > upper_boundary, df_train_str_sub_med_[col].median(), 
                        np.where(df_train_str_sub_med_[col] < lower_boundary, df_train_str_sub_med_[col].median(), df_train_str_sub_med_[col]))
# cохраним результат в новую переменную
df_train_str_sub_med_QT = df_train_str_sub_med_

In [65]:
# выделим целевой признак в отдельную переменную после обработки выбросов
y_train_str_sub_med_QT = df_train_str_sub_med_QT.pop('Прочность при растяжении, МПа')
X_train_str_sub_med_QT = df_train_str_sub_med_QT

# формируем DataFrame на основе массива
y_train_str_sub_med_QT = pd.DataFrame(y_train_str_sub_med_QT, columns=['Прочность при растяжении, МПа'])